<a href="https://colab.research.google.com/github/sidharthdk/BMS-Data-pre-processing-files/blob/main/SNL_NMC_BATTERY_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# === SNL NMC BATTERY DATASET: LOCAL ZIP → CLEANED ZIP ===
# Upload your SNL NMC ZIP to Colab Files panel, then RUN this

from google.colab import files
import zipfile
import pandas as pd
import numpy as np
from pathlib import Path

# Step 1: Upload ZIP
print("📤 Upload your SNL NMC battery dataset ZIP:")
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]
print(f"✅ Processing: {zip_path}")

# Step 2: Extract to temp dir
raw_dir = Path("/content/snl_nmc_raw")
raw_dir.mkdir(exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(raw_dir)
print(f"Extracted to: {raw_dir}")

csv_files = list(raw_dir.rglob("*.csv"))
print(f"Found {len(csv_files)} CSV files")

# Step 3: SNL NMC config (18650 NMC cells, typical 3.0-4.2V)[web:73][web:82]
CONFIG = {
    'interp_gap': 10,
    'v_min': 3.0,          # NMC lower cutoff (SNL tests)
    'v_max': 4.2,          # NMC upper cutoff
    'drop_bad_cycles': True
}

clean_dir = Path("/content/snl_nmc_cleaned")
clean_dir.mkdir(exist_ok=True)

def clean_snl_nmc_file(path: Path) -> bool:
    """Clean SNL NMC CSV (handles cycle_data & timeseries)"""
    fname = path.name
    print(f"  Cleaning {fname}...")
    try:
        df = pd.read_csv(path)
        if df.empty:
            print("    ⚠ Empty file")
            return False
        orig_rows = len(df)

        # 1) Standardize columns (SNL/BatteryML format)
        df.columns = (
            df.columns.str.strip().str.lower()
            .str.replace(r'[^a-z0-9_]', '', regex=True)
            .str.replace(r'_+', '_', regex=True)
        )

        # SNL column standardization
        rename_map = {
            'cycle': 'cycle_index', 'cycleid': 'cycle_index',
            'test_time_s': 'test_time_s', 'time_s': 'test_time_s',
            'voltage_v': 'voltage_v', 'v': 'voltage_v', 'voltage': 'voltage_v',
            'current_a': 'current_a', 'i': 'current_a', 'current': 'current_a',
            'temperature_c': 'temperature_c', 'temp': 'temperature_c',
            'discharge_capacity_ah': 'discharge_capacity_ah',
            'charge_capacity_ah': 'charge_capacity_ah'
        }
        df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns}, inplace=True)

        # 2) Numeric conversion & basic cleaning
        df = df.apply(pd.to_numeric, errors='coerce')
        df = df.dropna(axis=1, how='all').dropna(how='all')
        df = df.drop_duplicates()

        # 3) Time-based interpolation (per cycle)
        num_cols = df.select_dtypes(np.number).columns
        if len(num_cols) > 0 and len(df) > 1:
            time_col = next((c for c in ['test_time_s', 'time_s'] if c in df), None)
            if time_col:
                sort_cols = ['cycle_index', time_col] if 'cycle_index' in df else [time_col]
                df = df.sort_values(sort_cols)

            df[num_cols] = df[num_cols].interpolate(
                method='linear', limit=CONFIG['interp_gap'], limit_direction='both'
            )
            df[num_cols] = df[num_cols].ffill().bfill()
            df = df.dropna(subset=num_cols)

        # 4) NMC physics constraints (SNL 18650 specs)[web:73]
        if 'voltage_v' in df:
            df = df[(df['voltage_v'] >= CONFIG['v_min']) & (df['voltage_v'] <= CONFIG['v_max'])]

        # Capacity/energy non-negative
        for col in num_cols:
            if any(x in col for x in ['capacity', 'energy', 'ah', 'wh']):
                df[col] = df[col].clip(lower=0)

        # 5) Monotonic time validation per cycle
        if CONFIG['drop_bad_cycles'] and 'cycle_index' in df and time_col in df:
            is_valid_cycle = df.groupby('cycle_index')[time_col].apply(
                lambda x: x.is_monotonic_increasing
            )
            bad_cycles = is_valid_cycle[~is_valid_cycle].index
            if len(bad_cycles) > 0:
                df = df[~df['cycle_index'].isin(bad_cycles)]
                print(f"    ⚠ Dropped {len(bad_cycles)} bad cycles")

        # 6) Save viable results
        if len(df) < 10:
            print(f"    ⚠ Too small after cleaning ({len(df)} rows)")
            return False

        out_name = f"{path.stem}_cleaned.csv"
        out_path = clean_dir / out_name
        df.to_csv(out_path, index=False)

        retention = len(df) / orig_rows * 100
        print(f"    ✅ {len(df):,} rows ({retention:.0f}% retention)")
        return True

    except Exception as e:
        print(f"    ❌ {type(e).__name__}: {str(e)[:60]}")
        return False

# Process all files
processed = 0
for i, csv_file in enumerate(csv_files):
    print(f"[{i+1}/{len(csv_files)}]", end=" ")
    if clean_snl_nmc_file(csv_file):
        processed += 1

print(f"\n{'='*60}")
print(f"🎉 SNL NMC CLEANING COMPLETE: {processed}/{len(csv_files)} files")
print(f"Raw input size: {sum(f.stat().st_size for f in csv_files)/1e6:.1f} MB")

# Step 7: Create output ZIP
output_zip = "/content/snl_nmc_battery_cleaned.zip"
with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
    for cleaned_file in clean_dir.glob("*.csv"):
        zf.write(cleaned_file, cleaned_file.name)

print(f"\n📦 Output ZIP: {output_zip}")
!du -sh {output_zip} {clean_dir}

# Quick validation
if list(clean_dir.glob("*.csv")):
    sample = pd.read_csv(list(clean_dir.glob("*.csv"))[0])
    print(f"\n📊 Sample file shape: {sample.shape}")
    print(f"Columns: {list(sample.columns)}")
    if 'voltage_v' in sample:
        print(f"Voltage range: {sample['voltage_v'].min():.2f}V - {sample['voltage_v'].max():.2f}V ✓")


📤 Upload your SNL NMC battery dataset ZIP:


Saving SNL NMC-20260122T061645Z-1-001.zip to SNL NMC-20260122T061645Z-1-001.zip
✅ Processing: SNL NMC-20260122T061645Z-1-001.zip
Extracted to: /content/snl_nmc_raw
Found 64 CSV files
[1/64]   Cleaning SNL_18650_NMC_25C_40-60_0.5-3C_b_timeseries.csv...
    ✅ 344,109 rows (100% retention)
[2/64]   Cleaning SNL_18650_NMC_35C_0-100_0.5-1C_c_cycle_data.csv...
    ✅ 784 rows (100% retention)
[3/64]   Cleaning SNL_18650_NMC_25C_0-100_0.5-3C_a_cycle_data.csv...
    ✅ 729 rows (100% retention)
[4/64]   Cleaning SNL_18650_NMC_25C_0-100_0.5-1C_b_cycle_data.csv...
    ✅ 784 rows (100% retention)
[5/64]   Cleaning SNL_18650_NMC_25C_0-100_0.5-1C_d_timeseries.csv...
    ✅ 74,811 rows (100% retention)
[6/64]   Cleaning SNL_18650_NMC_25C_20-80_0.5-3C_b_timeseries.csv...
    ✅ 83,010 rows (100% retention)
[7/64]   Cleaning SNL_18650_NMC_15C_0-100_0.5-1C_b_cycle_data.csv...
    ✅ 512 rows (100% retention)
[8/64]   Cleaning SNL_18650_NMC_35C_0-100_0.5-2C_a_timeseries.csv...
    ✅ 96,586 rows (100% retenti